<a href="https://colab.research.google.com/github/leeje008/PyTorch/blob/master/%EC%9B%94%EB%B6%80_DA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 환경 세팅



### 필요한 패키지 설치

In [1]:
!pip install dash dash-bootstrap-components pandas sqlalchemy pymysql

### 사전 작업 수행

- 제공받은 데이터의 로컬 다운로드 및 구글 드라이브 업로드

In [2]:
# 라이브러리 로드

import sqlite3
import dash
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
from sqlalchemy import create_engine
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')

# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# working directory 변경 (해당 경로에 데이터 업로드)
path = '/content/drive/MyDrive/DA_work/'
os.chdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 경로 확인
os.getcwd()

'/content/drive/MyDrive/DA_work'

In [4]:
# SQLite 데이터베이스 파일 생성 (파일이 없으면 자동 생성)
db_path = 'example.db'  # 데이터베이스 파일 이름
conn = sqlite3.connect(path + db_path)
cursor = conn.cursor()

## 문제 1
샘플 데이터를 참조해서, BA를 위한 테이블을 생성하는 SQL 코드를 작성하시오.

In [5]:
# 첫 번째 테이블 생성 (예시)
cursor.execute("""
CREATE TABLE customer (
    customer_id INTEGER PRIMARY KEY,
    age INTEGER,
    gender VARCHAR(10),
    region VARCHAR(30),
    ad_consent VARCHAR(10),
    signup_path VARCHAR(10)
);
""")

# 두 번째 테이블 생성
cursor.execute("""
CREATE TABLE category (
    product_id INTEGER PRIMARY KEY,
    "category" VARCHAR(50),
    product_name VARCHAR(50),
    price INTEGER
);
""")

# 네 번째 테이블 생성
cursor.execute("""
CREATE TABLE coupon (
    coupon_name VARCHAR(20) PRIMARY KEY,
    discount_amount VARCHAR(20)
);
""")

# 세 번째 테이블 생성 (order의 경우 예약어로 월부_오더의 약자인 wb_order로 대체)
cursor.execute("""
CREATE TABLE "wb_order" (
    order_date DATE,
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    product_id INTEGER,
    period INTEGER,
    coupon_name VARCHAR(20),
    FOREIGN KEY (customer_id) REFERENCES customer_id(customer_id),
    FOREIGN KEY (product_id) REFERENCES category(product_id),
    FOREIGN KEY (coupon_name) REFERENCES coupon(coupon_name)
);
""")

## 문제 2.

대시보드 제작에 필요한 데이터를 조회하는 SQL 쿼리문을 작성하시오

### 대시보드 목록

#### Customer

#### Order

- 일별/월별 주문자 수
- 일별/월별 주문 강의 수
- 일별/월별 주문 수강 기간 (주문자가 신청한 수강 기간의 합)
- 일별/월별 주문금액

#### Category

- 일별/월별 강의 유형별 주문자 수
- 일별/월별 강의 유형별 주문 수강 기간 (주문자가 신청한 수강 기간의 합)
- 일별/월별 강의 유형별 주문금액

#### Coupon

- 일별/월별 쿠폰 할인 금액 (쿠폰 사용으로 할인받은 금액)
- 일별/월별 쿠폰 할인 금액 비율 (주문 금액 대비)
- 일별/월별 쿠폰 사용자 수
- 일별/월별 쿠폰 사용자 비율 (주문자 수 대비)



In [6]:
# 샘플 데이터 로드

customer = pd.read_csv('/content/drive/MyDrive/DA_work/월급쟁이부자들 DA 직무 과제 _샘플 데이터 - customer.csv')
order = pd.read_csv('/content/drive/MyDrive/DA_work/월급쟁이부자들 DA 직무 과제 _샘플 데이터 - order.csv')
category = pd.read_csv('/content/drive/MyDrive/DA_work/월급쟁이부자들 DA 직무 과제 _샘플 데이터 - category.csv')
coupon = pd.read_csv('/content/drive/MyDrive/DA_work/월급쟁이부자들 DA 직무 과제 _샘플 데이터 - coupon.csv')

# 문제 1번에서 생성한 테이블과 칼럼명을 일치시키기 위한 코드

order = order.rename(columns={'coupon name': 'coupon_name'})
coupon = coupon.rename(columns={'coupon name': 'coupon_name'})

In [7]:
# 데이터 삽입 함수
def insert_data(df, table_name):
    # 컬럼명 리스트 가져오기
    columns = df.columns.tolist()

    # 동적으로 쿼리문 작성
    column_names = ', '.join(columns)  # 컬럼명 (column1, column2, column3)
    placeholders = ', '.join(['?'] * len(columns))  # 값 자리표시자 (?, ?, ?)

    # SQL 쿼리문
    sql_query = f"""
    INSERT INTO {table_name} ({column_names})
    VALUES ({placeholders});
    """

    # 데이터 삽입
    for _, row in df.iterrows():
        cursor.execute(sql_query, tuple(row))


# 첫 번째 시트 데이터 삽입
insert_data(customer, 'customer')

# 두 번째 시트 데이터 삽입
insert_data(order, 'wb_order')

# 세 번째 시트 데이터 삽입
insert_data(category, 'category')

# 네 번째 시트 데이터 삽입
insert_data(coupon, 'coupon')

# 데이터베이스에 저장
conn.commit()

In [8]:
# 데이터 확인 함수
def check_data(table_name):
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()
    count = 0
    for row in rows:
        print(row)
        count += 1
        if count > 9:
          break

# 각 테이블의 데이터 확인
print("customer 테이블 데이터:")
check_data('customer')
print('-' * 40)

print("wb_order 테이블 데이터:")
check_data('wb_order')
print('-' * 40)

print("category 테이블 데이터:")
check_data('category')
print('-' * 40)

print("coupon 테이블 데이터:")
check_data('coupon')
print('-' * 40)

customer 테이블 데이터:
(101, 50, 'Male', 'Gyeonggi', 'Yes', 'google')
(102, 29, 'Female', 'Incheon', 'No', 'google')
(103, 22, 'Female', 'Jeollabuk', 'Yes', 'email')
(104, 44, 'Female', 'Gyeongsangnam', 'Yes', 'email')
(105, 26, 'Male', 'Seoul', 'No', 'naver')
(106, 51, 'Male', 'Ulsan', 'Yes', 'kakao')
(107, 21, 'Female', 'Gyeonggi', 'Yes', 'kakao')
(108, 28, 'Female', 'Incheon', 'Yes', 'google')
(109, 22, 'Female', 'Incheon', 'Yes', 'email')
(110, 30, 'Female', 'Gangwon', 'No', 'naver')
----------------------------------------
wb_order 테이블 데이터:
('2022. 4. 1', 1, 101, 14, 2, None)
('2022. 4. 1', 2, 102, 10, 3, 'no.5')
('2022. 4. 1', 3, 103, 9, 1, None)
('2022. 4. 2', 4, 105, 25, 1, None)
('2022. 4. 3', 5, 106, 27, 2, 'no.6')
('2022. 4. 3', 6, 104, 23, 3, None)
('2022. 4. 3', 7, 109, 17, 5, None)
('2022. 4. 4', 8, 106, 22, 3, None)
('2022. 4. 4', 9, 107, 29, 1, 'no.4')
('2022. 4. 4', 10, 105, 30, 1, 'no.4')
----------------------------------------
category 테이블 데이터:
(1, '부동산', '열반기초', 200)
(2

In [9]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc


# 데이터 조회
cursor.execute("SELECT * FROM customer;")
rows = cursor.fetchall()

# customer DataFrame을 생성
sql_data = pd.DataFrame(rows, columns = customer.columns)

### Customer 대시보드

- 가입 경로별 회원 비율
- 마케팅 광고 수신 동의 여부별 회원 비율
- 지역별 회원 비율
- 성별별 회원 비율
- 연령별 회원 비율

In [10]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
# Dash 앱 초기화
app1 = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


# 데이터 조회
cursor.execute("SELECT * FROM customer;")
rows = cursor.fetchall()

# customer DataFrame을 생성
sql_data = pd.DataFrame(rows, columns = customer.columns)

# 카테고리별 시각화 생성
signup_fig = px.pie(sql_data, names='signup_path', title="가입 경로별 회원 비율")
ad_consent_fig = px.pie(sql_data, names='ad_consent', title="마케팅 광고 수신 동의 여부별 회원 비율")

region_fig  = px.pie(sql_data, names='region', title="지역별 회원 비율")
gender_fig  = px.pie(sql_data, names='gender', title="성별별 회원 비율")

sql_data['age_group'] = pd.cut(sql_data['age'],
                         bins=[20, 30, 40, 50, 55],  # 20~30, 30~40, 40~50, 50~55
                         right=False,               # 왼쪽 포함, 오른쪽 미포함
                         labels=["20대", "30대", "40대", "50대"])
sql_data

# 연령대별 비율 계산
age_group_counts = sql_data['age_group'].value_counts().sort_index()
age_group_percentages = (age_group_counts / len(sql_data)) * 100

age_fig = px.pie(
    values=age_group_counts,
    names=age_group_counts.index,
    title="연령대별 회원 비율",
    labels={'names': '연령대'}
)




region_counts = sql_data['region'].value_counts().reset_index()
region_counts.columns = ['region', 'count']  # 컬럼 이름을 region과 count로 설정


region_fig_bar =px.bar(region_counts, x='region', y='count', labels={'region': '지역', 'count': '회원 수'},
             title="지역별 회원 수 막대 그래프")

age_group_counts = sql_data['age_group'].value_counts().reset_index()
age_group_counts.columns = ['age_group', 'count']

# 트리맵 시각화
age_fig_tree = px.treemap(
    age_group_counts,
    path=['age_group'],       # 계층적 경로
    values='count',           # 각 그룹의 크기
    title="연령별 회원 비율 트리맵"
)


# Dash 레이아웃 구성
app1.layout = dbc.Container([
    html.H1("고객 데이터 대시보드", className="text-center mt-4 mb-4"),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=signup_fig), width=6),
        dbc.Col(dcc.Graph(figure=ad_consent_fig), width=6),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=region_fig), width=6),
        dbc.Col(dcc.Graph(figure=gender_fig), width=6),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=age_fig), width = 6),
        dbc.Col(dcc.Graph(figure=region_fig_bar), width = 6)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=age_fig_tree), width = 12)
    ])
], fluid=True)

# 앱 실행

app1.run_server(port=8050, debug=True, use_reloader=False)

<IPython.core.display.Javascript object>

In [11]:
# SQL 쿼리 실행
cursor.execute("""
SELECT
    o.order_date,
    o.order_id,
    o.customer_id,
    o.product_id,
    o.period,
    o.coupon_name,
    strftime('%m',
        printf('%04d-%02d-%02d',
            CAST(substr(o.order_date, 1, 4) AS INTEGER),
            CAST(substr(o.order_date, 7, 1) AS INTEGER),
            CAST(substr(o.order_date, 10, 2) AS INTEGER)
        )
    ) AS month,
    strftime('%d',
        printf('%04d-%02d-%02d',
            CAST(substr(o.order_date, 1, 4) AS INTEGER),
            CAST(substr(o.order_date, 7, 1) AS INTEGER),
            CAST(substr(o.order_date, 10, 2) AS INTEGER)
        )
    ) AS day,
    c.price  -- category 테이블의 price 칼럼만 추가
FROM wb_order o
LEFT JOIN category c
    ON o.product_id = c.product_id;
""")
rows = cursor.fetchall()

# 컬럼 이름을 정의
order_columns = ['order_date', 'order_id', 'customer_id', 'product_id',
                 'period', 'coupon_name', 'month', 'day', 'price']

# DataFrame 생성
order_data = pd.DataFrame(rows, columns=order_columns)

### Order 대시보드

- 일별/월별 주문자 수
- 일별/월별 주문 강의 수
- 일별/월별 주문 수강 기간 (주문자가 신청한 수강 기간의 합)
- 일별/월별 주문금액

In [12]:
# Dash 앱 초기화
app2 = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


# 일별 주문자 수
daily_customer_count = order_data.groupby('day')['customer_id'].nunique().reset_index()
daily_customer_count = daily_customer_count.rename(columns={'customer_id': 'daily_customer_count'})

# 월별 주문자 수
monthly_customer_count = order_data.groupby('month')['customer_id'].nunique().reset_index()
monthly_customer_count = monthly_customer_count.rename(columns={'customer_id': 'monthly_customer_count'})

# 일별 주문 강의 수
daily_product_count = order_data.groupby('day')['product_id'].nunique().reset_index()
daily_product_count = daily_product_count.rename(columns={'product_id': 'daily_product_count'})

# 월별 주문 강의 수
monthly_product_count = order_data.groupby('month')['product_id'].nunique().reset_index()
monthly_product_count = monthly_product_count.rename(columns={'product_id': 'monthly_product_count'})

# 일별 주문 수강 기간 합
daily_period_sum = order_data.groupby('day')['period'].sum().reset_index()
daily_period_sum = daily_period_sum.rename(columns={'period': 'daily_period_sum'})

# 월별 주문 수강 기간 합
monthly_period_sum = order_data.groupby('month')['period'].sum().reset_index()
monthly_period_sum = monthly_period_sum.rename(columns={'period': 'monthly_period_sum'})

# 일별 주문금액
daily_order_amount = order_data.groupby('day')['price'].sum().reset_index()
daily_order_amount = daily_order_amount.rename(columns={'price': 'daily_order_amount'})

# 월별 주문금액
monthly_order_amount = order_data.groupby('month')['price'].sum().reset_index()
monthly_order_amount = monthly_order_amount.rename(columns={'price': 'monthly_order_amount'})

# 시각화를 위한 Plotly 그래프 (일별/월별 시각화)
daily_customer_chart = px.line(daily_customer_count, x='day', y='daily_customer_count',
                               title='일별 주문자 수', labels={'day': '날짜', 'daily_customer_count': '주문자 수'})

monthly_customer_chart = px.line(monthly_customer_count, x='month', y='monthly_customer_count',
                                 title='월별 주문자 수', labels={'month': '월', 'monthly_customer_count': '주문자 수'})

daily_product_chart = px.line(daily_product_count, x='day', y='daily_product_count',
                              title='일별 주문 강의 수', labels={'day': '날짜', 'daily_product_count': '주문 강의 수'})

monthly_product_chart = px.line(monthly_product_count, x='month', y='monthly_product_count',
                                title='월별 주문 강의 수', labels={'month': '월', 'monthly_product_count': '주문 강의 수'})

daily_period_chart = px.line(daily_period_sum, x='day', y='daily_period_sum',
                             title='일별 주문 수강 기간 합', labels={'day': '날짜', 'daily_period_sum': '수강 기간 합'})

monthly_period_chart = px.line(monthly_period_sum, x='month', y='monthly_period_sum',
                               title='월별 주문 수강 기간 합', labels={'month': '월', 'monthly_period_sum': '수강 기간 합'})

daily_order_amount_chart = px.line(daily_order_amount, x='day', y='daily_order_amount',
                                   title='일별 주문금액', labels={'day': '날짜', 'daily_order_amount': '주문금액'})

monthly_order_amount_chart = px.line(monthly_order_amount, x='month', y='monthly_order_amount',
                                     title='월별 주문금액', labels={'month': '월', 'monthly_order_amount': '주문금액'})

# Dash 레이아웃 설정
app2.layout = html.Div([
    html.H1("주문 데이터 대시보드", className="text-center mt-4 mb-4"),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=daily_customer_chart), width=6),
        dbc.Col(dcc.Graph(figure=monthly_customer_chart), width=6),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=daily_product_chart), width=6),
        dbc.Col(dcc.Graph(figure=monthly_product_chart), width=6),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=daily_period_chart), width=6),
        dbc.Col(dcc.Graph(figure=monthly_period_chart), width=6),
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(figure=daily_order_amount_chart), width=6),
        dbc.Col(dcc.Graph(figure=monthly_order_amount_chart), width=6),
    ]),
])

app2.run_server(port=8051, debug=True, use_reloader=False)

<IPython.core.display.Javascript object>

In [13]:
cursor.execute("""
SELECT
    c.category,
    c.product_id,
    c.price,  -- category 테이블에서 price만 가져옴
    o.order_date,
    o.order_id,
    o.customer_id,
    o.period,
    o.coupon_name,
    strftime('%m',
        printf('%04d-%02d-%02d',
            CAST(substr(o.order_date, 1, 4) AS INTEGER),
            CAST(substr(o.order_date, 7, 1) AS INTEGER),
            CAST(substr(o.order_date, 10, 2) AS INTEGER)
        )
    ) AS month,
    strftime('%d',
        printf('%04d-%02d-%02d',
            CAST(substr(o.order_date, 1, 4) AS INTEGER),
            CAST(substr(o.order_date, 7, 1) AS INTEGER),
            CAST(substr(o.order_date, 10, 2) AS INTEGER)
        )
    ) AS day
FROM category c
LEFT JOIN wb_order o
    ON o.product_id = c.product_id;
""")

rows = cursor.fetchall()


category_columns = [description[0] for description in cursor.description]
category_data = pd.DataFrame(rows, columns=category_columns)

In [14]:
# 'day' 열을 기준으로 집계
customer_count_daily = category_data.groupby(['day', 'category'])['customer_id'].nunique().reset_index(name='customer_count')
course_duration_daily = category_data.groupby(['day', 'category'])['period'].sum().reset_index(name='total_duration')
order_amount_daily = category_data.groupby(['day', 'category'])['price'].sum().reset_index(name='total_amount')

# 월별 데이터 집계 (기존대로 'month' 기준)
customer_count_monthly = category_data.groupby(['month', 'category'])['customer_id'].nunique().reset_index(name='customer_count')
course_duration_monthly = category_data.groupby(['month', 'category'])['period'].sum().reset_index(name='total_duration')
order_amount_monthly = category_data.groupby(['month', 'category'])['price'].sum().reset_index(name='total_amount')

### Category 대시보드

- 일별/월별 강의 유형별 주문자 수
- 일별/월별 강의 유형별 주문 수강 기간 (주문자가 신청한 수강 기간의 합)
- 일별/월별 강의 유형별 주문금액

In [15]:
import dash
from dash import dcc, html
import plotly.express as px

# Dash 앱 초기화
app3 = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# 일별 데이터 시각화
fig_customer_count_daily = px.line(customer_count_daily, x='day', y='customer_count', color='category')
fig_course_duration_daily = px.line(course_duration_daily, x='day', y='total_duration', color='category')
fig_order_amount_daily = px.line(order_amount_daily, x='day', y='total_amount', color='category')

# 월별 데이터 시각화
fig_customer_count_monthly = px.line(customer_count_monthly, x='month', y='customer_count', color='category')
fig_course_duration_monthly = px.line(course_duration_monthly, x='month', y='total_duration', color='category')
fig_order_amount_monthly = px.line(order_amount_monthly, x='month', y='total_amount', color='category')

# 대시보드 레이아웃
app3.layout = html.Div([
    html.H1("강의 유형별 주문 분석 대시보드"),

    html.Div([
        html.H2("일별 강의 유형별 주문자 수"),
        dcc.Graph(figure=fig_customer_count_daily)
    ], style={'padding': '10px'}),

    html.Div([
        html.H2("일별 강의 유형별 수강 기간 합"),
        dcc.Graph(figure=fig_course_duration_daily)
    ], style={'padding': '10px'}),

    html.Div([
        html.H2("일별 강의 유형별 주문금액"),
        dcc.Graph(figure=fig_order_amount_daily)
    ], style={'padding': '10px'}),

    html.Div([
        html.H2("월별 강의 유형별 주문자 수"),
        dcc.Graph(figure=fig_customer_count_monthly)
    ], style={'padding': '10px'}),

    html.Div([
        html.H2("월별 강의 유형별 수강 기간 합"),
        dcc.Graph(figure=fig_course_duration_monthly)
    ], style={'padding': '10px'}),

    html.Div([
        html.H2("월별 강의 유형별 주문금액"),
        dcc.Graph(figure=fig_order_amount_monthly)
    ], style={'padding': '10px'})
])


app3.run_server(port=8052, debug=True, use_reloader=False)

<IPython.core.display.Javascript object>

In [16]:
cursor.execute("""
SELECT
    o.order_date,
    o.order_id,
    o.customer_id,
    o.product_id,
    o.period,
    o.coupon_name,
    strftime('%m',
        printf('%04d-%02d-%02d',
            CAST(substr(o.order_date, 1, 4) AS INTEGER),
            CAST(substr(o.order_date, 7, 1) AS INTEGER),
            CAST(substr(o.order_date, 10, 2) AS INTEGER)
        )
    ) AS month,
    strftime('%d',
        printf('%04d-%02d-%02d',
            CAST(substr(o.order_date, 1, 4) AS INTEGER),
            CAST(substr(o.order_date, 7, 1) AS INTEGER),
            CAST(substr(o.order_date, 10, 2) AS INTEGER)
        )
    ) AS day,
    c.price,  -- category 테이블의 price 칼럼
    cp.discount_amount  -- coupon 테이블의 discount_amount 칼럼
FROM "wb_order" o
LEFT JOIN category c
    ON o.product_id = c.product_id  -- product_id를 기준으로 category 테이블과 join
LEFT JOIN coupon cp
    ON o.coupon_name = cp.coupon_name;  -- coupon_name을 기준으로 coupon 테이블과 join
""")

rows = cursor.fetchall()

# 컬럼 이름을 가져오는 부분 수정
coupon_columns = [description[0] for description in cursor.description]

# 데이터프레임 생성
coupon_data = pd.DataFrame(rows, columns=coupon_columns)


In [17]:
# 할인 금액 계산 함수
def calculate_discount_amount(row):
    price = row['price']
    discount_amount = row['discount_amount']

    if discount_amount is None:
        return 0  # 할인 금액이 없으면 0 반환
    elif isinstance(discount_amount, str) and '%' in discount_amount:
        # 할인 금액이 %로 제공되면 %를 제외하고 계산
        discount_percentage = float(discount_amount.replace('%', ''))
        discount = (discount_percentage * price) / 100
    else:
        # 할인 금액이 숫자일 경우 (정액 할인 금액)
        discount = float(discount_amount)

    # 할인 금액이 price를 초과하면 price만큼만 할인
    if discount > price:
        return price
    return discount

coupon_data['discount_amount_calculated'] = coupon_data.apply(calculate_discount_amount, axis=1)

### 쿠폰 대시보드

- 일별/월별 쿠폰 할인 금액 (쿠폰 사용으로 할인받은 금액)
- 일별/월별 쿠폰 할인 금액 비율 (주문 금액 대비)
- 일별/월별 쿠폰 사용자 수
- 일별/월별 쿠폰 사용자 비율 (주문자 수 대비)

In [18]:
coupon_data

,order_date,order_id,customer_id,product_id,period,coupon_name,month,day,price,discount_amount,discount_amount_calculated
0,2022. 4. 1,1,101,14,2,None,04,01,200,None,0.0
1,2022. 4. 1,2,102,10,3,no.5,04,01,300,10%,30.0
2,2022. 4. 1,3,103,9,1,None,04,01,250,None,0.0
3,2022. 4. 2,4,105,25,1,None,04,02,600,None,0.0
4,2022. 4. 3,5,106,27,2,no.6,04,03,250,15%,37.5
...,...,...,...,...,...,...,...,...,...,...,...
924,2022. 9. 15,925,292,27,1,no.5,09,15,250,10%,25.0
925,2022. 9. 15,926,116,21,1,None,09,15,200,None,0.0
926,2022. 9. 15,927,298,10,2,None,09,15,300,None,0.0
927,2022. 9. 15,928,294,18,5,no.5,09,15,600,10%,60.0


In [19]:
# 일별/월별 쿠폰 할인 금액 계산
coupon_data['coupon_used'] = coupon_data['coupon_name'].notnull()  # 쿠폰 사용 여부

# 할인 금액 비율 계산
coupon_data['discount_percentage'] = coupon_data.apply(
    lambda row: (row['discount_amount_calculated'] / row['price']) * 100 if row['price'] != 0 else 0, axis=1
)

# 일별과 월별로 집계
coupon_discount_day = coupon_data.groupby(['day']).agg(
    total_discount=('discount_amount_calculated', 'sum'),
    total_price=('price', 'sum'),
    total_users=('coupon_used', 'sum'),
    total_orders=('order_id', 'count')
).reset_index()

coupon_discount_month = coupon_data.groupby(['month']).agg(
    total_discount=('discount_amount_calculated', 'sum'),
    total_price=('price', 'sum'),
    total_users=('coupon_used', 'sum'),
    total_orders=('order_id', 'count')
).reset_index()

# 할인 금액 비율 계산 (price와 discount_amount의 총합 비율)
coupon_discount_day['discount_percentage'] = (coupon_discount_day['total_discount'] / coupon_discount_day['total_price']) * 100
coupon_discount_month['discount_percentage'] = (coupon_discount_month['total_discount'] / coupon_discount_month['total_price']) * 100

# 쿠폰 사용자 비율 계산
coupon_discount_day['user_percentage'] = (coupon_discount_day['total_users'] / coupon_discount_day['total_orders']) * 100
coupon_discount_month['user_percentage'] = (coupon_discount_month['total_users'] / coupon_discount_month['total_orders']) * 100

# 색상 구분을 위해 추가 컬럼 생성 (전체 대비 일부 비율을 구분)
coupon_discount_day['discount_color'] = coupon_discount_day['discount_percentage'].apply(
    lambda x: 'Total' if x >= 50 else 'Partial'
)
coupon_discount_month['discount_color'] = coupon_discount_month['discount_percentage'].apply(
    lambda x: 'Total' if x >= 50 else 'Partial'
)

# Dash 앱 초기화
app4 = dash.Dash(__name__)

# 일별/월별 그래프 (색상 구분 적용)
fig1_day = px.bar(coupon_discount_day, x='day', y='total_discount', color='discount_color', title="일별 쿠폰 할인 금액")
fig2_day = px.bar(coupon_discount_day, x='day', y='discount_percentage', color='discount_color', title="일별 쿠폰 할인 금액 비율")
fig3_day = px.bar(coupon_discount_day, x='day', y='total_users', title="일별 쿠폰 사용자 수")
fig4_day = px.bar(coupon_discount_day, x='day', y='user_percentage', title="일별 쿠폰 사용자 비율")

fig1_month = px.bar(coupon_discount_month, x='month', y='total_discount', color='discount_color', title="월별 쿠폰 할인 금액")
fig2_month = px.bar(coupon_discount_month, x='month', y='discount_percentage', color='discount_color', title="월별 쿠폰 할인 금액 비율")
fig3_month = px.bar(coupon_discount_month, x='month', y='total_users', title="월별 쿠폰 사용자 수")
fig4_month = px.bar(coupon_discount_month, x='month', y='user_percentage', title="월별 쿠폰 사용자 비율")

# 레이아웃 정의
app4.layout = html.Div([
    html.H1("쿠폰 사용 대시보드"),

    # 일별 그래프
    html.H3("일별 쿠폰 할인 금액"),
    dcc.Graph(figure=fig1_day),
    html.H3("일별 쿠폰 할인 금액 비율"),
    dcc.Graph(figure=fig2_day),
    html.H3("일별 쿠폰 사용자 수"),
    dcc.Graph(figure=fig3_day),
    html.H3("일별 쿠폰 사용자 비율"),
    dcc.Graph(figure=fig4_day),

    # 월별 그래프
    html.H3("월별 쿠폰 할인 금액"),
    dcc.Graph(figure=fig1_month),
    html.H3("월별 쿠폰 할인 금액 비율"),
    dcc.Graph(figure=fig2_month),
    html.H3("월별 쿠폰 사용자 수"),
    dcc.Graph(figure=fig3_month),
    html.H3("월별 쿠폰 사용자 비율"),
    dcc.Graph(figure=fig4_month, style={'height': '80vh', 'width': '80vw'}),
])

# 서버 실행
app4.run_server(port=8053, debug=True, use_reloader=False)

<IPython.core.display.Javascript object>

### 문제 3번